<a href="https://colab.research.google.com/github/SandeshRangreji/Pointer-Generator-Networks/blob/main/DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import tensorflow_datasets as tfds
import tensorflow as tf
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# unzips Glove word embeddings
!unrar x "/content/drive/MyDrive/Pointer Generator Networks/glove.6B.200d.rar" -d "/content/PGN/data/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open /content/drive/MyDrive/Pointer Generator Networks/glove.6B.200d.rar
No such file or directory
No files to extract


In [12]:
# class to handle data loading, splitting, preprocessing, tokenization
class Data:

  def __init__(self):
    self.contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

  def split_contractions(self, sentence):
    li_sentence = sentence.split(' ')
    for i in range(len(li_sentence)):
      li_sentence[i] = self.contractions.get(li_sentence[i], li_sentence[i])
    sentence = ' '.join(li_sentence)
    return sentence

  # function to handle preprocessing of articles and summaries
  def preprocess(self, sentence):
    # parameters:
    # sentence: article or summary to be processed
    # returns:
    # sentence: cleaned article/summary
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    sentence = self.split_contractions(sentence)
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    # removing trailing spaces
    sentence = sentence.strip()
    return sentence  
  
  # splitting data into articles and summaries for training and testing
  def split_data(self, dataset):
    # parameters:
    # dataset : tfds of cnn_dailymail dataset (bytes)
    # returns:
    # train_articles, train_summaries, eval_articles, eval_summaries : lists of training and eval articles and summaries (string)
    train_articles = []
    train_summaries = []
    eval_articles = []
    eval_summaries = []
    # iterating through train dataset and storing articles and summaries seperately
    for text in tfds.as_numpy(dataset['train']):
      # decoding from bytes to string
      article = self.preprocess(text['article'].decode("utf-8"))
      summaries = self.preprocess(text['highlights'].decode("utf-8"))
      train_articles.append(article)
      train_summaries.append(summaries)
    # iterating through validation dataset and storing articles and summaries seperately
    for text in tfds.as_numpy(dataset['validation']):
      # decoding from bytes to string
      article = self.preprocess(text['article'].decode("utf-8"))
      summaries = self.preprocess(text['highlights'].decode("utf-8"))
      eval_articles.append(article)
      eval_summaries.append(summaries)
    return train_articles, train_summaries, eval_articles, eval_summaries

  # function to tokenize data
  def tokenize(self, train_articles, train_summaries, eval_articles, eval_summaries, vocab_size , embedding_dim, max_length_articles, max_length_summaries, truncating_type, padding_type, oov_token):
    # parameters:
    # train_articles, train_summaries, eval_articles, eval_summaries : lists of training and eval articles and summaries (string)
    # vocab_size: size of vocabulary
    # embedding_dim: dimensions of word embeddings
    # max_lengths_articles: number of words in the longest article
    # max_lengths_summaries: number of words in the longest summary
    # truncating_type: pre/post truncatation
    # padding_type: pro/post padding
    # oov_token: specifies what oov_token should be used
    # return:
    # train_articles, train_summaries, eval_articles, eval_summaries : lists of training and eval articles and summaries (sequences)
    # initialize tokenizer
    tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
    # fit tokenizer on training input (vocab)
    tokenizer.fit_on_texts(train_articles)
    # get word index from tokenizer
    word_index = tokenizer.word_index
    # tokenize articles for training 
    train_articles = tokenizer.texts_to_sequences(train_articles)
    train_articles = pad_sequences(train_articles ,maxlen=max_length_articles, padding=padding_type, truncating=truncating_type)
    # tokenize summaries for training 
    eval_articles = tokenizer.texts_to_sequences(eval_articles)
    eval_articles = pad_sequences(eval_articles, maxlen=max_length_articles, padding=padding_type, truncating=truncating_type)
    # tokenize articles for eval 
    train_summaries = tokenizer.texts_to_sequences(train_summaries)
    train_summaries = pad_sequences(train_summaries ,maxlen=max_length_articles, padding=padding_type, truncating=truncating_type)
    # tokenize summaries for eval 
    eval_summaries = tokenizer.texts_to_sequences(eval_summaries)
    eval_summaries = pad_sequences(eval_summaries, maxlen=max_length_articles, padding=padding_type, truncating=truncating_type)
    return train_articles, train_summaries, eval_articles, eval_summaries, word_index

  # function to get a dictionery which is the reverse of the word_index
  def get_reverse_word_index(self, word_index):
    # parameters:
    # word_index : { word : id }
    # returns
    # reverse_word_index : { id : word }
    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    return reverse_word_index

  def __call__(self, vocab_size = 50000, embedding_dim = 200, max_length_articles = 16000, max_length_summaries = 8000, truncating_type='post', padding_type='post', oov_token='<OOV>'):
    # parameters:
    # vocab_size: size of vocabulary
    # embedding_dim: dimensions of word embeddings
    # max_lengths_articles: number of words in the longest article
    # max_lengths_summaries: number of words in the longest summary
    # truncating_type: pre/post truncatation
    # padding_type: pro/post padding
    # oov_token: specifies what oov_token should be used
    # returns:
    # train_articles, train_summaries, eval_articles, eval_summaries : lists of training and eval articles and summaries (sequences)
    # word_index : { word : id }
    # reverse_word_index : { id : word }

    # loading data in bytes from tfds
    ds=tfds.load("cnn_dailymail")
    # splitting data into train and test sets of articles and summaries
    train_articles, train_summaries, eval_articles, eval_summaries = self.split_data(ds)
    # tokenizing data
    train_articles, train_summaries, eval_articles, eval_summaries, word_index = self.tokenize(
        train_articles[:500], 
        train_summaries[:500], 
        eval_articles[:500], 
        eval_summaries[:500],
        vocab_size , 
        embedding_dim, 
        max_length_articles, 
        max_length_summaries, 
        truncating_type, 
        padding_type, 
        oov_token)
    return train_articles, train_summaries, eval_articles, eval_summaries, word_index, self.get_reverse_word_index(word_index)

In [13]:
data = Data()
vocab_size = 10000
embedding_dim = 200
max_length_articles = 16000
max_length_summaries = 8000
truncating_type ='post'
padding_type ='post'
oov_token = "<OOV>"
train_articles, train_summaries, eval_articles, eval_summaries, word_index, reverse_word_index = data(
        vocab_size, 
        embedding_dim, 
        max_length_articles, 
        max_length_summaries, 
        truncating_type, 
        padding_type, 
        oov_token)

In [14]:
word_index

{'<OOV>': 1,
 'the': 2,
 'to': 3,
 'a': 4,
 'and': 5,
 'of': 6,
 'in': 7,
 's': 8,
 'was': 9,
 'that': 10,
 'for': 11,
 'is': 12,
 'on': 13,
 'he': 14,
 'it': 15,
 'with': 16,
 'said': 17,
 'his': 18,
 'i': 19,
 'as': 20,
 'at': 21,
 'have': 22,
 'not': 23,
 'by': 24,
 'from': 25,
 'be': 26,
 'has': 27,
 'are': 28,
 'her': 29,
 'but': 30,
 'they': 31,
 'she': 32,
 'an': 33,
 'who': 34,
 'this': 35,
 'had': 36,
 'after': 37,
 'we': 38,
 'their': 39,
 'been': 40,
 'were': 41,
 'will': 42,
 'when': 43,
 'you': 44,
 'one': 45,
 'which': 46,
 'up': 47,
 'about': 48,
 'out': 49,
 'would': 50,
 'year': 51,
 'more': 52,
 'there': 53,
 'also': 54,
 'or': 55,
 'people': 56,
 'so': 57,
 'all': 58,
 'can': 59,
 'two': 60,
 'him': 61,
 'what': 62,
 'if': 63,
 'into': 64,
 'first': 65,
 'new': 66,
 'them': 67,
 'time': 68,
 'do': 69,
 'just': 70,
 'over': 71,
 'no': 72,
 'police': 73,
 'last': 74,
 'than': 75,
 'could': 76,
 'told': 77,
 'mr': 78,
 'my': 79,
 'years': 80,
 'before': 81,
 'being': 82

In [15]:
reverse_word_index

{1: '<OOV>',
 2: 'the',
 3: 'to',
 4: 'a',
 5: 'and',
 6: 'of',
 7: 'in',
 8: 's',
 9: 'was',
 10: 'that',
 11: 'for',
 12: 'is',
 13: 'on',
 14: 'he',
 15: 'it',
 16: 'with',
 17: 'said',
 18: 'his',
 19: 'i',
 20: 'as',
 21: 'at',
 22: 'have',
 23: 'not',
 24: 'by',
 25: 'from',
 26: 'be',
 27: 'has',
 28: 'are',
 29: 'her',
 30: 'but',
 31: 'they',
 32: 'she',
 33: 'an',
 34: 'who',
 35: 'this',
 36: 'had',
 37: 'after',
 38: 'we',
 39: 'their',
 40: 'been',
 41: 'were',
 42: 'will',
 43: 'when',
 44: 'you',
 45: 'one',
 46: 'which',
 47: 'up',
 48: 'about',
 49: 'out',
 50: 'would',
 51: 'year',
 52: 'more',
 53: 'there',
 54: 'also',
 55: 'or',
 56: 'people',
 57: 'so',
 58: 'all',
 59: 'can',
 60: 'two',
 61: 'him',
 62: 'what',
 63: 'if',
 64: 'into',
 65: 'first',
 66: 'new',
 67: 'them',
 68: 'time',
 69: 'do',
 70: 'just',
 71: 'over',
 72: 'no',
 73: 'police',
 74: 'last',
 75: 'than',
 76: 'could',
 77: 'told',
 78: 'mr',
 79: 'my',
 80: 'years',
 81: 'before',
 82: 'being'